# EA3 - Analisis no supervisado con K-Means (scoring de default)

Trabajo alineado a CRISP-DM para complementar el modelo supervisado existente. Solo se usa el set de entrenamiento (data/train.csv) para evitar data leakage. La variable default solo se emplea para interpretar segmentos, no para entrenar K-Means.


## 0. Setup
- Importar librerias y configurar graficos.
- Cargar data/train.csv como df_train. Si el archivo no esta disponible se genera un dataset sintetico para poder ejecutar el flujo.
- Mostrar head(), info() y conteo de nulos.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

rng = np.random.default_rng(42)
data_path = Path("data/train.csv")

if data_path.exists():
    df_train = pd.read_csv(data_path)
    source_msg = "Dataset cargado desde data/train.csv"
else:
    n_samples = 800
    df_train = pd.DataFrame({
        "age": rng.normal(45, 10, n_samples).clip(18, 80),
        "income": rng.lognormal(mean=10.5, sigma=0.5, size=n_samples),
        "loan_amount": rng.normal(120000, 30000, n_samples).clip(20000, 300000),
        "num_late_payments": rng.poisson(1.2, n_samples),
        "credit_score": rng.normal(650, 50, n_samples).clip(300, 850),
    })
    df_train["loan_income_ratio"] = df_train["loan_amount"] / df_train["income"]
    logits = (
        -6
        + 0.8 * df_train["loan_income_ratio"]
        + 0.6 * (df_train["num_late_payments"] > 0)
        - 0.003 * df_train["credit_score"]
    )
    probs = 1 / (1 + np.exp(-logits))
    df_train["default"] = rng.binomial(1, probs.clip(0, 1))
    source_msg = "Se genero un dataset sintetico para poder ejecutar el flujo (reemplazar con data real en data/train.csv)."

print(source_msg)
df_train.head()


In [ ]:
df_train.info()
df_train.isnull().sum()


## 1. Data Understanding
- Seleccionar variables numericas relevantes para el clustering (sin la columna default).
- Entender su comportamiento con estadisticas descriptivas.


In [ ]:
candidate_cols = [
    "age",
    "income",
    "loan_amount",
    "loan_income_ratio",
    "num_late_payments",
    "debt_to_income",
    "credit_score",
]
available = [c for c in candidate_cols if c in df_train.columns]
other_numeric = [
    c
    for c in df_train.select_dtypes(include="number").columns
    if c not in available and c != "default"
]
num_cols = available if available else other_numeric
if not num_cols:
    raise ValueError("No hay columnas numericas disponibles para clustering.")

X = df_train[num_cols].copy()
print("Columnas usadas para clustering:", num_cols)
X.describe()


## 2. Data Preparation
- Manejo de nulos solo dentro del set de entrenamiento (imputacion con la media por columna numerica).
- Escalamiento con StandardScaler porque K-Means usa distancia euclidiana sensible a la escala de las variables.


In [ ]:
X_prepared = X.copy()
missing = X_prepared.isnull().sum()
if missing.sum() > 0:
    X_prepared = X_prepared.fillna(X_prepared.mean())
    print("Se imputaron nulos con la media en columnas:", missing[missing > 0].index.tolist())
else:
    print("No se encontraron nulos en las columnas seleccionadas.")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_prepared)
X_scaled_df = pd.DataFrame(X_scaled, columns=num_cols)
X_scaled_df.head()


## 3. Eleccion de numero de clusters (metodo del codo)
- Se prueba K en el rango [2, 8].
- Se grafica la inercia (SSE) para identificar un codo donde la reduccion marginal de error se estabiliza.


In [ ]:
inertia = []
k_range = range(2, 9)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.plot(list(k_range), inertia, marker="o")
plt.title("Metodo del codo")
plt.xlabel("Numero de clusters (k)")
plt.ylabel("Inercia (SSE)")
plt.xticks(list(k_range))
plt.show()


### Interpretacion del codo
Seleccionar un valor de k donde la curva pierde pendiente marcada (ejemplo: 3 o 4). Se fija k_opt manualmente con base en la grafica y conocimiento del negocio.


## 4. Entrenamiento de K-Means
- Ajuste del modelo con el k_opt elegido.
- Se agrega la etiqueta cluster al DataFrame de entrenamiento. No se utilizan datos de validacion o test.


In [ ]:
k_opt = 3  # ajustar si la grafica del codo sugiere otro valor
kmeans = KMeans(n_clusters=k_opt, random_state=42, n_init=10)

df_train["cluster"] = kmeans.fit_predict(X_scaled)
df_train["cluster"].value_counts().sort_index()


## 5. Analisis de clusters
- Perfil promedio por cluster en variables numericas.
- Tasa de default por cluster (solo para evaluar, no usada en el entrenamiento no supervisado).


In [ ]:
cluster_profile = df_train.groupby("cluster")[num_cols].mean()
cluster_profile


In [ ]:
if "default" in df_train.columns:
    cluster_default_rate = df_train.groupby("cluster")["default"].mean()
    print("Tasa de default por cluster:")
    display(cluster_default_rate)
else:
    print("Columna 'default' no encontrada; saltando tasa de default.")


In [ ]:
if "default" in df_train.columns:
    ax = df_train.groupby("cluster")["default"].mean().plot(kind="bar", color="steelblue")
    ax.set_ylabel("Tasa de default")
    ax.set_xlabel("Cluster")
    ax.set_title("Tasa de default promedio por cluster")
    plt.xticks(rotation=0)
    plt.show()


## 6. Interpretacion de clusters
Describir los perfiles observados (ejemplo):
- Cluster 0: mayor relacion deuda/ingreso y mas mora -> riesgo mas alto.
- Cluster 1: ingresos medios y morosidad moderada.
- Cluster 2: ingresos altos, pocas moras -> riesgo bajo.

El analisis complementa el modelo supervisado al revelar subsegmentos donde conviene ajustar politicas (precio, campanas, limites de credito) y priorizar monitoreo.


## 7. Conclusiones finales
- K-Means permite segmentar clientes sin usar la etiqueta default, evitando data leakage.
- La etiqueta cluster puede evaluarse como feature adicional en el modelo supervisado (reentrenar y validar antes de usar en produccion).
- Considerar recalibrar periodicamente, inicializaciones distintas y evaluar otras tecnicas (silhouette, DBSCAN, clustering jerarquico) en el futuro.
